In [10]:
import pandas as pd
from tqdm import tqdm
from nixtla import NixtlaClient

In [15]:
nixtla_client = NixtlaClient(
    api_key = 'nixtla-tok-H4du2F5cFbD3n0BghkN98TeC1PfhHbtXdNIRWSZOVNauP55C4Vs4sVuYDb3XTmiuoYRdYhm7k0AiUbdZ' # misaki api
)
nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [17]:
start_point = 2070*12
batch_size = 12
num = 30
output_file = './data/forecasts_2071_3000.csv'

flatten_df = pd.read_csv('./data/flattened_loop_sensor_test_12h.csv')
flatten_df = flatten_df.iloc[start_point :start_point + batch_size * num]
flatten_df['t_1h'] = pd.to_datetime(flatten_df['t_1h'])
print(flatten_df.shape)
flatten_df.head()

(360, 5)


,id,iu_ac,t_1h,etat_barre,q
24840,2071,11,2023-11-11 16:00:00,3,830.0
24841,2071,11,2023-11-11 17:00:00,3,847.0
24842,2071,11,2023-11-11 18:00:00,3,863.0
24843,2071,11,2023-11-11 19:00:00,3,797.0
24844,2071,11,2023-11-11 20:00:00,3,865.0


In [18]:
num_batches = len(flatten_df) // batch_size
all_forecasts = []

In [20]:
# Forecasting loop
for i in tqdm(range(num_batches), desc='Forecasting'):
    # 读取12条数据
    df = flatten_df.iloc[i * batch_size:(i + 1) * batch_size]
    
    # 进行预测
    timegpt_fcst_df = nixtla_client.forecast(
        df=df, 
        h=1,  # 预测步长
        # finetune_steps=10,
        # finetune_loss='mae',   
        freq='h', 
        time_col='t_1h', 
        target_col='q', 
        model='timegpt-1'
    )

    # Add index starting from 1 + start_point
    timegpt_fcst_df.reset_index(drop=True, inplace=True)
    timegpt_fcst_df.index += 1 + int(start_point/12) + i

    # Save the current forecast to the output CSV file
    if i == 0:
        # For the first batch, write the header
        timegpt_fcst_df.to_csv(output_file, index=True, index_label='id', mode='w')
    else:
        # For subsequent batches, append without the header
        timegpt_fcst_df.to_csv(output_file, index=True, index_label='id', mode='a', header=False)

    # Append the current forecast to the list
    all_forecasts.append(timegpt_fcst_df)

# Optionally, you can still combine all forecasts into one DataFrame and perform additional operations if needed
all_forecasts_df = pd.concat(all_forecasts, ignore_index=False)
print(all_forecasts_df.shape)
print(all_forecasts_df.head())


Forecasting:   0%|          | 0/30 [00:00<?, ?it/s]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   3%|▎         | 1/30 [00:04<02:12,  4.58s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   7%|▋         | 2/30 [00:08<01:52,  4.01s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:  10%|█         | 3/30 [00:10<01:25,  3.16s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nix

(60, 2)
                       t_1h     TimeGPT
2071.0  2023-11-12 04:00:00  677.903809
2072.0  2023-11-13 09:00:00  713.562378
2073.0  2023-11-14 14:00:00  767.833801
2074.0  2023-11-15 19:00:00  885.833984
2075.0           2023-11-17  831.326050
